In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [2]:
RANDOM_STATE = 42

results_regression = pd.DataFrame(columns = ['model', 'task', 'R2'])
results_classification = pd.DataFrame(columns = ['model', 'task', 'f1', 'accuracy'])

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%, предварительно выделив целевую переменную (колонка 'MEDV').

In [4]:
y = data['MEDV']
X = data.drop('MEDV', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = RANDOM_STATE)

2. Обучите стандартную регрессию, а также Ridge и  Lasso с параметрами по умолчанию и выведите их R2 на тестовой выборке

In [5]:
r2_lr = 0
r2_ridge = 0
r2_lasso = 0

lr = LinearRegression()
lr.fit(X_train, y_train)

ridge = Ridge()
ridge.fit(X_train, y_train)

lasso = Lasso()
lasso.fit(X_train, y_train)

r2_lr = r2_score(y_test, lr.predict(X_test))
r2_ridge = r2_score(y_test, ridge.predict(X_test))
r2_lasso = r2_score(y_test, lasso.predict(X_test))



results_regression.loc[0] = ['LR', 'task2', r2_lr]
results_regression.loc[1] = ['Ridge', 'task2', r2_ridge]
results_regression.loc[2] = ['Lasso', 'task2', r2_lasso]

print(results_regression.loc[0])
print(results_regression.loc[1])
print(results_regression.loc[2])


model          LR
task        task2
R2       0.668483
Name: 0, dtype: object
model       Ridge
task        task2
R2       0.665961
Name: 1, dtype: object
model       Lasso
task        task2
R2       0.666869
Name: 2, dtype: object


3. Для Ridge и Lasso подберите коэффициент регуляризации двумя способами 1) GridSearchCV, 2) RidgeCV и LassoCV, в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по всем моделям и сравните с предыдущими результатами.

In [6]:


r2_ridge_grid_search = 0
r2_ridge_cv = 0
r2_lasso_grid_search = 0
r2_lasso_cv = 0

# Ridge с использованием GridSearchCV
ridge = Ridge()
ridge_params = {'alpha': np.logspace(-5, 5, 11)}
ridge_grid = GridSearchCV(ridge, ridge_params, cv=5)
ridge_grid.fit(X_train, y_train)
ridge_pred = ridge_grid.predict(X_test)
ridge_r2 = r2_score(y_test, ridge_pred)

# Lasso с использованием GridSearchCV
lasso = Lasso()
lasso_params = {'alpha': np.logspace(-5, 5, 11)}
lasso_grid = GridSearchCV(lasso, lasso_params, cv=5)
lasso_grid.fit(X_train, y_train)
lasso_pred = lasso_grid.predict(X_test)
lasso_r2 = r2_score(y_test, lasso_pred)

# Ridge с использованием RidgeCV
ridge_cv = RidgeCV(alphas=np.logspace(-5, 5, 11), cv=5)
ridge_cv.fit(X_train, y_train)
ridge_cv_pred = ridge_cv.predict(X_test)
ridge_cv_r2 = r2_score(y_test, ridge_cv_pred)


# Lasso с использованием LassoCV
lasso_cv = LassoCV(alphas=np.logspace(-5, 5, 11), cv=5)
lasso_cv.fit(X_train, y_train)
lasso_cv_pred = lasso_cv.predict(X_test)
lasso_cv_r2 = r2_score(y_test, lasso_cv_pred)

# Выводим результаты


print('R2 для Ridge с GridSearchCV:', ridge_r2)
print('R2 для Ridge с RidgeCV:', ridge_cv_r2)
print('R2 для Lasso с GridSearchCV:', lasso_r2)
print('R2 для Lasso с LassoCV:', lasso_cv_r2)


R2 для Ridge с GridSearchCV: 0.6684825680074256
R2 для Ridge с RidgeCV: 0.6684825680074256
R2 для Lasso с GridSearchCV: 0.6684829595885677
R2 для Lasso с LassoCV: 0.6684829595885677


<i>По сравнению спредыдущими результат примерно одинаковый</i>

4. Проведите масштабирование выборки (используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [7]:
# Pipeline для масштабирования данных
scaler = StandardScaler()
pipe = Pipeline([('scaler', scaler), ('ridge', Ridge())])

# Ridge с использованием параметров по умолчанию и масштабированием данных
pipe.fit(X_train, y_train)
ridge_pred = pipe.predict(X_test)
ridge_r2 = r2_score(y_test, ridge_pred)

# Lasso с использованием параметров по умолчанию и масштабированием данных
pipe = Pipeline([('scaler', scaler), ('lasso', Lasso())])
pipe.fit(X_train, y_train)
lasso_pred = pipe.predict(X_test)
lasso_r2 = r2_score(y_test, lasso_pred)

# Выводим результаты
print('R2 для Ridge с параметрами по умолчанию и масштабированием данных:', ridge_r2)
print('R2 для Lasso с параметрами по умолчанию и масштабированием данных:', lasso_r2)

R2 для Ridge с параметрами по умолчанию и масштабированием данных: 0.668190107677443
R2 для Lasso с параметрами по умолчанию и масштабированием данных: 0.624044752347846


<i>По сравнению спредыдущими результат примерно одинаковый</i>

In [8]:
# StandardScaler Масштабирование данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Ridge с использованием параметров по умолчанию и масштабированием данных
ridge_scaled = Ridge()
ridge_scaled.fit(X_train_scaled, y_train)
ridge_pred_scaled = ridge_scaled.predict(X_test_scaled)
ridge_r2_scaled = r2_score(y_test, ridge_pred_scaled)

# Lasso с использованием параметров по умолчанию и масштабированием данных
lasso_scaled = Lasso()
lasso_scaled.fit(X_train_scaled, y_train)
lasso_pred_scaled = lasso_scaled.predict(X_test_scaled)
lasso_r2_scaled = r2_score(y_test, lasso_pred_scaled)

# Выводим результаты
print('R2 для Ridge с параметрами по умолчанию и масштабированием данных:', ridge_r2_scaled)
print('R2 для Lasso с параметрами по умолчанию и масштабированием данных:', lasso_r2_scaled)

R2 для Ridge с параметрами по умолчанию и масштабированием данных: 0.668190107677443
R2 для Lasso с параметрами по умолчанию и масштабированием данных: 0.624044752347846


<i>По сравнению спредыдущими результат примерно одинаковый</i>

In [9]:
# MinMaxScaler Масштабирование данных
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Ridge с использованием параметров по умолчанию и масштабированием данных
ridge_scaled = Ridge()
ridge_scaled.fit(X_train_scaled, y_train)
ridge_pred_scaled = ridge_scaled.predict(X_test_scaled)
ridge_r2_scaled = r2_score(y_test, ridge_pred_scaled)

# Lasso с использованием параметров по умолчанию и масштабированием данных
lasso_scaled = Lasso()
lasso_scaled.fit(X_train_scaled, y_train)
lasso_pred_scaled = lasso_scaled.predict(X_test_scaled)
lasso_r2_scaled = r2_score(y_test, lasso_pred_scaled)

# Выводим результаты
print('R2 для Ridge с параметрами по умолчанию и масштабированием данных:', ridge_r2_scaled)
print('R2 для Lasso с параметрами по умолчанию и масштабированием данных:', lasso_r2_scaled)

R2 для Ridge с параметрами по умолчанию и масштабированием данных: 0.6762207658974596
R2 для Lasso с параметрами по умолчанию и масштабированием данных: 0.2573921442545195


<i>По сравнению спредыдущими результат хуже</i>

5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами.

In [10]:
# Масштабирование данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Ridge с использованием GridSearchCV и масштабированием данных
ridge = Ridge()
ridge_params = {'alpha': np.logspace(-5, 5, 11)}
ridge_grid = GridSearchCV(ridge, ridge_params, cv=5)
ridge_grid.fit(X_train_scaled, y_train)
ridge_pred = ridge_grid.predict(X_test_scaled)
ridge_r2 = r2_score(y_test, ridge_pred)

# Lasso с использованием GridSearchCV и масштабированием данных
lasso = Lasso()
lasso_params = {'alpha': np.logspace(-5, 5, 11)}
lasso_grid = GridSearchCV(lasso, lasso_params, cv=5)
lasso_grid.fit(X_train_scaled, y_train)
lasso_pred = lasso_grid.predict(X_test_scaled)
lasso_r2 = r2_score(y_test, lasso_pred)

# Выводим результаты
print('R2 для Ridge с GridSearchCV и масштабированием данных:', ridge_r2)
print('R2 для Lasso с GridSearchCV и масштабированием данных:', lasso_r2)

R2 для Ridge с GridSearchCV и масштабированием данных: 0.668190107677443
R2 для Lasso с GridSearchCV и масштабированием данных: 0.6684821312777706


<i>По сравнению спредыдущими результат примерно одинаковый(кроме худшего результата)</i>

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [11]:
# Добавление попарных произведений признаков и их квадратов
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

# Ridge с использованием параметров по умолчанию и масштабированием данных
ridge_poly = Ridge()
ridge_poly.fit(X_train_poly, y_train)
ridge_pred_poly = ridge_poly.predict(X_test_poly)
ridge_r2_poly = r2_score(y_test, ridge_pred_poly)

# Lasso с использованием параметров по умолчанию и масштабированием данных
lasso_poly = Lasso()
lasso_poly.fit(X_train_poly, y_train)
lasso_pred_poly = lasso_poly.predict(X_test_poly)
lasso_r2_poly = r2_score(y_test, lasso_pred_poly)

# Выводим результаты
print('R2 для Ridge с параметрами по умолчанию и масштабированием данных:', ridge_r2_poly)
print('R2 для Lasso с параметрами по умолчанию и масштабированием данных:', lasso_r2_poly)



R2 для Ridge с параметрами по умолчанию и масштабированием данных: 0.8171359789657097
R2 для Lasso с параметрами по умолчанию и масштабированием данных: 0.7322738282708989


<i>По сравнению спредыдущими результат лучше</i>

7. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, добавив PolynomialFeatures, посчитайте R2 и сравните с предыдущими результатами.

In [12]:

# Добавление попарных произведений признаков и их квадратов
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

# Ridge с использованием GridSearchCV и масштабированием данных
ridge = Ridge()
ridge_params = {'alpha': np.logspace(-5, 5, 11)}
ridge_grid = GridSearchCV(ridge, ridge_params, cv=5)
ridge_grid.fit(X_train_poly, y_train)
ridge_pred = ridge_grid.predict(X_test_poly)
ridge_r2 = r2_score(y_test, ridge_pred)

# Lasso с использованием GridSearchCV и масштабированием данных
lasso = Lasso()
lasso_params = {'alpha': np.logspace(-5, 5, 11)}
lasso_grid = GridSearchCV(lasso, lasso_params, cv=5)
lasso_grid.fit(X_train_poly, y_train)
lasso_pred = lasso_grid.predict(X_test_poly)
lasso_r2 = r2_score(y_test, lasso_pred)

# Выводим результаты
print('R2 для Ridge с GridSearchCV и масштабированием данных:', ridge_r2)
print('R2 для Lasso с GridSearchCV и масштабированием данных:', lasso_r2)

c:\Users\AVV\Documents\Programming\Python\Teach\School_DS\code\teach-scool-ds\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.156e+02, tolerance: 2.882e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\AVV\Documents\Programming\Python\Teach\School_DS\code\teach-scool-ds\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.170e+02, tolerance: 2.764e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\AVV\Documents\Programming\Python\Teach\School_DS\code\teach-scool-ds\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective 

R2 для Ridge с GridSearchCV и масштабированием данных: 0.8187344606117988
R2 для Lasso с GridSearchCV и масштабированием данных: 0.8128138856150267


<i>По сравнению спредыдущими результат ещё лучше</i>

8. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2.

In [13]:
# Масштабирование данных
scaler = StandardScaler()
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)

# Создание пайплайна
pipe = Pipeline([('scaler', scaler), ('poly', poly), ('model', Ridge())])

# Подбор параметров с использованием GridSearchCV
param_grid = {
    'model': [Ridge(), Lasso()],
    'scaler': [StandardScaler(), MinMaxScaler()],
    'poly__degree': [1, 2, 3],
    'model__alpha': np.logspace(-5, 5, 11)
}
grid_search = GridSearchCV(pipe, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Вывод наилучших параметров
print('Наилучшие параметры:', grid_search.best_params_)

# Предсказание на тестовых данных
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Вычисление R2 на тестовой выборке
r2 = r2_score(y_test, y_pred)
print('R2 на тестовой выборке:', r2)


c:\Users\AVV\Documents\Programming\Python\Teach\School_DS\code\teach-scool-ds\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.158e+02, tolerance: 2.882e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\AVV\Documents\Programming\Python\Teach\School_DS\code\teach-scool-ds\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.169e+02, tolerance: 2.764e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\AVV\Documents\Programming\Python\Teach\School_DS\code\teach-scool-ds\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective 

Наилучшие параметры: {'model': Lasso(), 'model__alpha': np.float64(0.001), 'poly__degree': 3, 'scaler': MinMaxScaler()}
R2 на тестовой выборке: 0.8458384498865369


c:\Users\AVV\Documents\Programming\Python\Teach\School_DS\code\teach-scool-ds\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.790e+02, tolerance: 3.510e+00
  model = cd_fast.enet_coordinate_descent(


http://archive.ics.uci.edu/ml/datasets/Adult

In [4]:
data = pd.read_csv('adult.csv')
data.head(2)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K


In [5]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

9. Разделите выборку на признаки и целевую переменную(колонка class). Замените целевую переменную на числовые значения ('<=50K' - 1, '>50K' - 0).

In [17]:
# Разделение выборки на признаки и целевую переменную
X = data.drop('class', axis=1)
y = data['class']

# Замена целевой переменной на числовые значения
y = y.replace({'<=50K': 1, '>50K': 0})
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = RANDOM_STATE)

C:\Users\AVV\AppData\Local\Temp\ipykernel_13928\2409519010.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = y.replace({'<=50K': 1, '>50K': 0})


In [18]:
y

0        1
1        1
2        1
3        1
4        1
        ..
48837    1
48838    1
48839    1
48840    1
48841    0
Name: class, Length: 48842, dtype: int64

10. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [7]:
# Предсказание самого частого класса
y_pred_most_common = np.full(y_test.shape, y_train.mode()[0])

# Вычисление метрик
accuracy = accuracy_score(y_test, y_pred_most_common)
f1 = f1_score(y_test, y_pred_most_common)

# Вывод результатов
print('Accuracy:', accuracy)
print('F1-score:', f1)

Accuracy: 0.7589313133381104
F1-score: 0.8629459349356923


11. Выясните, присутствуют ли в данных пропуски. Если присутствуют, заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [8]:
# Проверка наличия пропусков
if X.isnull().sum().all() > 0:
    # Заполнение пропусков самыми частыми значениями
    imputer = SimpleImputer(strategy='most_frequent')
    X = imputer.fit_transform(X)


12. Выберите колонки с числовыми и категориальными переменными (используя возможности pandas).

In [22]:
# X.dtypes

In [9]:
# Разделение данных на числовые и категориальные признаки
num_cols = X.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X.select_dtypes(include=['object']).columns

13. Создайте пайплайн по обработке числовых и категориальных значений колонок (используйте OneHotEncoder,MinMaxScaler) и посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [10]:
# Создание пайплайна для числовых признаков
num_pipe = Pipeline([('scaler', MinMaxScaler())])

# Создание пайплайна для категориальных признаков
cat_pipe = Pipeline([('encoder', OneHotEncoder(handle_unknown='ignore'))])

# Создание общего пайплайна
preprocessor = ColumnTransformer([
    ('num', num_pipe, num_cols),
    ('cat', cat_pipe, cat_cols)
])

# Создание пайплайна с моделью
pipe = Pipeline([('preprocessor', preprocessor), 
                ('classifier', LogisticRegression())])

# Вычисление cross_val_score
scores = cross_val_score(pipe, X, y, cv=5, scoring='accuracy')
print('Accuracy:', scores.mean())

scores = cross_val_score(pipe, X, y, cv=5, scoring='f1_macro')
print('F1-score:', scores.mean())

# Повторите шаги для других моделей
pipe = Pipeline([('preprocessor', preprocessor), 
                ('classifier', KNeighborsClassifier())])
scores = cross_val_score(pipe, X, y, cv=5, scoring='accuracy')
print('Accuracy:', scores.mean())

scores = cross_val_score(pipe, X, y, cv=5, scoring='f1_macro')
print('F1-score:', scores.mean())

pipe = Pipeline([('preprocessor', preprocessor), 
                ('classifier', LinearSVC())])
scores = cross_val_score(pipe, X, y, cv=5, scoring='accuracy')
print('Accuracy:', scores.mean())

scores = cross_val_score(pipe, X, y, cv=5, scoring='f1_macro')
print('F1-score:', scores.mean())

Accuracy: 0.850825215215122
F1-score: 0.7802688335226533
Accuracy: 0.8247614719848316
F1-score: 0.7485462924696753
Accuracy: 0.8528725977953127
F1-score: 0.7820239749363405


14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями, (испольуйте SimpleImputer). Посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [41]:
# Разделение выборки на признаки и целевую переменную
X = data.drop('class', axis=1)
y = data['class']

# Замена целевой переменной на числовые значения
y = y.replace({'<=50K': 1, '>50K': 0})
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = RANDOM_STATE)

X = X.replace('?', None)
# X.head(2)

C:\Users\AVV\AppData\Local\Temp\ipykernel_13928\1634644355.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = y.replace({'<=50K': 1, '>50K': 0})


In [40]:
# Проверка наличия пропусков
if X.isnull().sum().sum() > 0:
    # Заполнение пропусков самыми частыми значениями
    imputer = SimpleImputer(strategy='most_frequent')
    X = imputer.fit_transform(X)
X = pd.DataFrame(X)
X.head(2)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States


In [42]:
num_cols = X.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X.select_dtypes(include=['object']).columns
# Создание пайплайна для числовых признаков
num_pipe = Pipeline([('scaler', MinMaxScaler())])

# Создание пайплайна для категориальных признаков
cat_pipe = Pipeline([('encoder', OneHotEncoder(handle_unknown='ignore'))])

# Создание общего пайплайна
preprocessor = ColumnTransformer([
    ('num', num_pipe, num_cols),
    ('cat', cat_pipe, cat_cols)
])

# Создание пайплайна с моделью
pipe = Pipeline([('preprocessor', preprocessor), 
                ('classifier', LogisticRegression())])

# Вычисление cross_val_score
scores = cross_val_score(pipe, X, y, cv=5, scoring='accuracy')
print('Accuracy:', scores.mean())

scores = cross_val_score(pipe, X, y, cv=5, scoring='f1_macro')
print('F1-score:', scores.mean())

# Повторите шаги для других моделей
pipe = Pipeline([('preprocessor', preprocessor), 
                ('classifier', KNeighborsClassifier())])
scores = cross_val_score(pipe, X, y, cv=5, scoring='accuracy')
print('Accuracy:', scores.mean())

scores = cross_val_score(pipe, X, y, cv=5, scoring='f1_macro')
print('F1-score:', scores.mean())

pipe = Pipeline([('preprocessor', preprocessor), 
                ('classifier', LinearSVC())])
scores = cross_val_score(pipe, X, y, cv=5, scoring='accuracy')
print('Accuracy:', scores.mean())

scores = cross_val_score(pipe, X, y, cv=5, scoring='f1_macro')
print('F1-score:', scores.mean())

Accuracy: 0.850825215215122
F1-score: 0.7802688335226533
Accuracy: 0.8247614719848316
F1-score: 0.7485462924696753
Accuracy: 0.8529340207608198
F1-score: 0.7821075156689796


15. Посчитайте cross_val_score по тем же алгоритмам и метрикам, если просто удалить значения '?'.

In [37]:
# Разделение выборки на признаки и целевую переменную
X = data.drop('class', axis=1)
y = data['class']

# Замена целевой переменной на числовые значения
y = y.replace({'<=50K': 1, '>50K': 0})
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = RANDOM_STATE)

num_cols = X2.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X2.select_dtypes(include=['object']).columns

X2 = X2.replace('?', pd.NA)
X2 = X2.dropna()
# X2

C:\Users\AVV\AppData\Local\Temp\ipykernel_13928\3354176193.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = y.replace({'<=50K': 1, '>50K': 0})


In [38]:
# Создание пайплайна для числовых признаков
num_pipe = Pipeline([('scaler', MinMaxScaler())])

# Создание пайплайна для категориальных признаков
cat_pipe = Pipeline([('encoder', OneHotEncoder(handle_unknown='ignore'))])

# Создание общего пайплайна
preprocessor = ColumnTransformer([
    ('num', num_pipe, num_cols),
    ('cat', cat_pipe, cat_cols)
])

# Создание пайплайна с моделью
pipe = Pipeline([('preprocessor', preprocessor), 
                ('classifier', LogisticRegression())])

# Вычисление cross_val_score
scores = cross_val_score(pipe, X, y, cv=5, scoring='accuracy')
print('Accuracy:', scores.mean())

scores = cross_val_score(pipe, X, y, cv=5, scoring='f1_macro')
print('F1-score:', scores.mean())

# Повторите шаги для других моделей
pipe = Pipeline([('preprocessor', preprocessor), 
                ('classifier', KNeighborsClassifier())])
scores = cross_val_score(pipe, X, y, cv=5, scoring='accuracy')
print('Accuracy:', scores.mean())

scores = cross_val_score(pipe, X, y, cv=5, scoring='f1_macro')
print('F1-score:', scores.mean())

pipe = Pipeline([('preprocessor', preprocessor), 
                ('classifier', LinearSVC())])
scores = cross_val_score(pipe, X, y, cv=5, scoring='accuracy')
print('Accuracy:', scores.mean())

scores = cross_val_score(pipe, X, y, cv=5, scoring='f1_macro')
print('F1-score:', scores.mean())

Accuracy: 0.850825215215122
F1-score: 0.7802688335226533
Accuracy: 0.8247614719848316
F1-score: 0.7485462924696753
Accuracy: 0.8528725977953127
F1-score: 0.7820239749363405


 16. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier на данных с замененными значениями '?' на самые частые значения.

In [39]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# Создание пайплайна с моделью
pipe = Pipeline([('preprocessor', preprocessor), 
                    ('classifier', RandomForestClassifier())])

# Вычисление cross_val_score
scores = cross_val_score(pipe, X, y, cv=5, scoring='accuracy')
print('Accuracy:', scores.mean())

scores = cross_val_score(pipe, X, y, cv=5, scoring='f1_macro')
print('F1-score:', scores.mean())

# Повторите шаги для GradientBoostingClassifier
pipe = Pipeline([('preprocessor', preprocessor), 
                    ('classifier', GradientBoostingClassifier())])
scores = cross_val_score(pipe, X, y, cv=5, scoring='accuracy')
print('Accuracy:', scores.mean())

scores = cross_val_score(pipe, X, y, cv=5, scoring='f1_macro')
print('F1-score:', scores.mean())

Accuracy: 0.8520945470172616
F1-score: 0.7880840816614416
Accuracy: 0.8675731196536806
F1-score: 0.801489222950938


17. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [44]:

best_params = {'Accuracy': 0.8675731196536806,
'F1-score': 0.801489222950938}
print('Параметры лучшей модели:\n', best_params)
f1_best = 0
acc_best = 0
results_classification.loc[12] = ['Best_Model', 'task17', f1_best, acc_best]

Параметры лучшей модели:
 {'Accuracy': 0.8675731196536806, 'F1-score': 0.801489222950938}


In [ ]:
results_classification